<a href="https://colab.research.google.com/github/FBmaia/projeto_analise_de_qualidade_de_dados/blob/main/analise_qualidade_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avaliação de qualidade dos dados

### Importação das bibliotecas necessárias para a análise.

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as dt

### Permitindo acesso ao Google Drive
Como estou utilizando o Google Colab para a criação do script de análise, o código permite o acesso a arquivos do meu Google Drive, onde se encontra a base de dados ***Case DQA.csv***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Criação de um DataFrame Pandas a partir do arquivo CSV

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Case Neoway/Base de dados - Case DQA.csv')

Verificação dos tipos de dados das colunas

In [ ]:
df.head()

,ID Processo,Número do Processo,Nome Autor,Nome Réu,Data Ajuizamento,Valor da Causa,Data da Sentença,Valor da Sentença,Tipo de Ação,Motivo da Ação,Advogado Autor,Advogado Réu,Resultado do Processo,Status,Vara
0,1,fg-37913-42,Edward Murphy,NaN,2024-04-26,NaN,2024-09-06,24240.0,Âçãõ Pènàl,Divórcio,Dana Kelly,NaN,Indefinido,Julgado,2ª Vara Cível
1,2,Lh-36673-92,Frank Williams,NaN,2023-11-19,"1.398,02750",2024-09-18,NaN,Ação Cível,Îndénizâçã,Ashley Chavez,NaN,Improcedente,Julgado,5ª Vara Cível
2,3,ej-52718-58,Clifford Adams,NaN,2022-01-28,NaN,NaN,NaN,Ação Trabalhista,Divórcio,Wendy Perez,NaN,Indefinido,Julgado,4ª Vara Cível
3,4,nA-99129-59,Donna Andrews,NaN,2024-02-25,NaN,2023-08-07,18702.0,Ação Trabalhista,Responsabilidade Civil,David Vargas,NaN,Arquivado,Em andamento,4ª Vara Cível
4,5,ay-51526-16,Melissa Scott,Scott Sheppard,2023-03-20,rrrr,2024-05-05,10155.0,Ação Penal,Responsabilidade Civil,Kenneth Flores,Valerie Davis,Arquivado,Julgado,2ª Vara Cível


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49970 entries, 0 to 49969
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID Processo            49970 non-null  int64 
 1   Número do Processo     49970 non-null  object
 2   Nome Autor             49970 non-null  object
 3   Nome Réu               25062 non-null  object
 4   Data Ajuizamento       49970 non-null  object
 5   Valor da Causa         24984 non-null  object
 6   Data da Sentença       25174 non-null  object
 7   Valor da Sentença      24992 non-null  object
 8   Tipo de Ação           49970 non-null  object
 9   Motivo da Ação         49970 non-null  object
 10  Advogado Autor         49970 non-null  object
 11  Advogado Réu           25062 non-null  object
 12  Resultado do Processo  49970 non-null  object
 13  Status                 49970 non-null  object
 14  Vara                   49970 non-null  object
dtypes: int64(1), object

## Avaliação de preenchimento dos dados

In [ ]:
# Calculando a porcentagem de valores não nulos e não vazios em cada coluna, arredondando para duas casas decimais para facilitar visualização.
percentual_preenchimento = ((df.notnull() & df.applymap(lambda x: str(x).strip() != "")).mean() * 100).round(2)

# Transformando o resultado em um Dataframe
df_preenchimento = percentual_preenchimento.reset_index()
df_preenchimento.columns = ['Coluna', 'Percentual de Preenchimento (%)']

print(df_preenchimento)

<ipython-input-6-1493a1c11498>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  percentual_preenchimento = ((df.notnull() & df.applymap(lambda x: str(x).strip() != "")).mean() * 100).round(2)


                   Coluna  Percentual de Preenchimento (%)
0             ID Processo                           100.00
1      Número do Processo                           100.00
2              Nome Autor                           100.00
3                Nome Réu                            50.15
4        Data Ajuizamento                           100.00
5          Valor da Causa                            50.00
6        Data da Sentença                            50.38
7       Valor da Sentença                            50.01
8            Tipo de Ação                           100.00
9          Motivo da Ação                           100.00
10         Advogado Autor                           100.00
11           Advogado Réu                            50.15
12  Resultado do Processo                           100.00
13                 Status                           100.00
14                   Vara                           100.00


## Avaliação de padronização dos dados

In [ ]:
def validar_padronizacao(df):
    resultados = {}

    # Cálculo do percentual de valores do campo "ID Processo" que são do tipo int
    resultados['ID Processo'] = df['ID Processo'].apply(lambda x: isinstance(x, int)).mean() * 100


    # Cálculo do percentual de valores do campo "Números do Processo" que possuem o formato "AB-12345-01", utilizando Regex
    regex_num_processo = r"^[a-zA-Z]{2}-\d{5}-\d{2}$"
    resultados['Número do Processo'] = df['Número do Processo'].apply(
        lambda x: bool(re.match(regex_num_processo, str(x)))
    ).mean() * 100


    # Cálculo do percentual de valores dos campos "Nome Autor", "Nome Réu", "Advogado Autor" e "Advogado Réu" que foram preenchidos com letras maiúsculas, sem números, caracteres especiais ou acentos
    # Utilizei Regex e um loop sobre todos os campos de nome da base para evitar a repetição da lógica
    def validar_nome(x):
        return bool(re.match(r"^[A-Z\s]+$", x)) if isinstance(x, str) else False

    for coluna in ['Nome Autor', 'Nome Réu', 'Advogado Autor', 'Advogado Réu']:
        resultados[coluna] = df[coluna].apply(validar_nome).mean() * 100


    # Cálculo do percentual de valores nos campos "Data Ajuizamento" e "Data da Sentença" que estão no formato AAAA-MM-DD
    def validar_data(data):
        try:
            return pd.to_datetime(data, format='%Y-%m-%d', errors='coerce') is not pd.NaT
        except:
            return False

    #Valida se a "Data da Sentença" está no formato AAAA-MM-DD
    resultados['Data Ajuizamento'] = df['Data Ajuizamento'].apply(
        lambda x: validar_data(x)
    ).mean() * 100

    # Valida se a "Data da Sentença" está no formato AAAA-MM-DD ou é None, conforme descrito no dicionário de dados
    resultados['Data da Sentença'] = df['Data da Sentença'].apply(
        lambda x: validar_data(x) if pd.notnull(x) else True
    ).mean() * 100


    # Cálculo do percentual de valores nos campos 'Valor da Causa' e 'Valor da Sentença' que estão em formato numérico com duas casas decimais
    # Utilizei Regex e um loop sobre todos os valores da base para evitar a repetição da lógica
    def validar_valor(x):
        return bool(re.match(r"^\d+(\.\d{2})?$", str(x))) if pd.notnull(x) else False

    for coluna in ['Valor da Causa', 'Valor da Sentença']:
        resultados[coluna] = df[coluna].apply(validar_valor).mean() * 100

    # Cálculo do percentual de valores no campo "Tipo de Ação" que possuem categorias válidas e estão preenchidos com letras maiúsculas, sem números, caracteres especiais ou acentos
    categorias_tipo_acao = ['AÇÃO PENAL', 'AÇÃO CÍVEL', 'AÇÃO TRABALHISTA']
    resultados['Tipo de Ação'] = df['Tipo de Ação'].apply(
        lambda x: x in categorias_tipo_acao if isinstance(x, str) else False
    ).mean() * 100

    # Cálculo do percentual de valores no campo "Motivo da Ação" que possuem categorias válidas e estão preenchidos com letras maiúsculas, sem números, caracteres especiais ou acentos
    categorias_motivo_acao = ['DIVÓRCIO', 'INDENIZAÇÃO', 'DÍVIDA']
    resultados['Motivo da Ação'] = df['Motivo da Ação'].apply(
        lambda x: x in categorias_motivo_acao if isinstance(x, str) else False
    ).mean() * 100

    # Cálculo do percentual de valores no campo "Resultado do Processo" que possuem categorias válidas e estão preenchidos com letras maiúsculas, sem números, caracteres especiais ou acentos
    categorias_resultado = ['ARQUIVADO', 'INDEFINIDO', 'IMPROCEDENTE', 'PROCEDENTE']
    resultados['Resultado do Processo'] = df['Resultado do Processo'].apply(
        lambda x: x in categorias_resultado if isinstance(x, str) else False
    ).mean() * 100

    # Cálculo do percentual de valores no campo "Status" que possuem categorias válidas e estão preenchidos com letras maiúsculas, sem números, caracteres especiais ou acentos
    categorias_status = ['JULGADO', 'EM ANDAMENTO']
    resultados['Status'] = df['Status'].apply(
        lambda x: x in categorias_status if isinstance(x, str) else False
    ).mean() * 100

    # Cálculo do percentual de valores do campo "Vara" que possuem categorias validas e estão preenchidos com letras maiúsculas e sem acentos
    categorias_vara = ['1ª VARA CIVEL', '2ª VARA CIVEL', '3ª VARA CRIMINAL','4ª VARA CIVEL', '5ª VARA CIVEL']
    resultados['Vara'] = df['Vara'].apply(
        lambda x: x in categorias_vara if isinstance(x, str) else False
    ).mean() * 100

    # Retorna o Dataframe com os resultados arredondados a duas casas decimais para facilitar a visualização
    return pd.DataFrame(resultados, index=['Percentual Padronizado']).T.round(2)

resultado_padronizacao = validar_padronizacao(df)

# Transformando o resultado em um Dataframe
df_padronizacao = resultado_padronizacao.reset_index()
df_padronizacao.columns = ['Coluna', 'Percentual Padronizado (%)']

print(df_padronizacao)

                   Coluna  Percentual Padronizado (%)
0             ID Processo                      100.00
1      Número do Processo                      100.00
2              Nome Autor                        0.00
3                Nome Réu                        0.00
4          Advogado Autor                        0.00
5            Advogado Réu                        0.00
6        Data Ajuizamento                       99.11
7        Data da Sentença                       99.75
8          Valor da Causa                        0.00
9       Valor da Sentença                        0.00
10           Tipo de Ação                        0.00
11         Motivo da Ação                        0.00
12  Resultado do Processo                        0.00
13                 Status                        0.00
14                   Vara                        0.00


## Avaliação de Consistência dos Dados

In [ ]:
def verificar_consistencia(df):
    inconsistencias = {}
    hoje = pd.Timestamp(dt.datetime.now())

    # Contagem dos casos em que a "Data Ajuizamento" não é anterior à "Data Sentença"
    inconsistencias['Data Ajuizamento maior que a Data da Sentença'] = df[
        pd.to_datetime(df['Data Ajuizamento'], errors='coerce') >
        pd.to_datetime(df['Data da Sentença'], errors='coerce')
    ].shape[0]

    return pd.DataFrame(inconsistencias, index=['Registros Incorretos']).T

resultado_consistencia = verificar_consistencia(df)

# Transformando o resultado em um Dataframe
df_consistencia = resultado_consistencia.reset_index()
df_consistencia.columns = ['Coluna', 'Contagem de registros']

print(df_consistencia)


                                          Coluna  Contagem de registros
0  Data Ajuizamento maior que a Data da Sentença                      6


## Avaliação de Unicidade dos Dados

In [ ]:
# Contagem das linhas duplicadas
duplicados = df.duplicated(keep=False)
resultado_duplicados = duplicados.sum()

# Transformando o resultado em um Dataframe
df_unicidade = pd.DataFrame({'Contagem de registros duplicados': [resultado_duplicados]})

print(df_unicidade)

   Contagem de registros duplicados
0                                 0


## Criação da planilha com todos os resultados para ser utilizada no relatório

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
writer = pd.ExcelWriter('avaliação_qualidade_de_dados.xlsx', engine='xlsxwriter')

df_preenchimento.to_excel(writer, sheet_name='Preenchimento')
df_padronizacao.to_excel(writer, sheet_name='Padronização')
df_consistencia.to_excel(writer, sheet_name='Consistência')
df_unicidade.to_excel(writer, sheet_name='Unicidade')

writer.close()